In [2]:
import pandas as pd
import numpy as np
import pickle
import sklearn
import tensorflow as tf
import torch
import torch.nn as nn
import torch.utils 
from torch.utils.data import Dataset,DataLoader
import torch.utils.data
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras import regularizers
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import mnist


In [3]:
dataset_path = "dataset_apoe4.pickle"
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(pd.__version__)
print(sklearn.__version__)


with open(dataset_path, 'rb') as f:
    X_train, y_train, X_test, y_test = pickle.load(f)


True
NVIDIA A100-SXM4-80GB
2.2.2
1.3.2


In [5]:
# Definition of the Autoencoder model as a subclass of the TensorFlow Model class

class SimpleAutoencoder(Model):
	def __init__(self,latent_dimensions , data_shape):
		super(SimpleAutoencoder, self).__init__()
		self.latent_dimensions = latent_dimensions
		self.data_shape = data_shape

		# Encoder architecture using a Sequential model
		self.encoder = tf.keras.Sequential([
			layers.Flatten(),
			layers.Dense(latent_dimensions, activation='relu'),
		])

		# Decoder architecture using another Sequential model
		self.decoder = tf.keras.Sequential([
			layers.Dense(tf.math.reduce_prod(data_shape), activation='sigmoid'),
			layers.Reshape(data_shape)
		])

	# Forward pass method defining the encoding and decoding steps
	def call(self, input_data):
		encoded_data = self.encoder(input_data)
		decoded_data = self.decoder(encoded_data)
		return decoded_data


In [6]:
# Extracting shape information from the testing dataset
input_data_shape = X_test.shape[1:]

# Specifying the dimensionality of the latent space
latent_dimensions = 64

# Creating an instance of the SimpleAutoencoder model
simple_autoencoder = SimpleAutoencoder(latent_dimensions, input_data_shape)

simple_autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

simple_autoencoder.fit(X_train, y_train,
				epochs=1,
				shuffle=True,
				validation_data=(X_test, y_test))


2024-07-15 18:47:09.161816: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 888 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:d0:00.0, compute capability: 8.0


ValueError: Exception encountered when calling SimpleAutoencoder.call().

[1mInvalid dtype: <property object at 0x7fa4cb599170>[0m

Arguments received by SimpleAutoencoder.call():
  • input_data=tf.Tensor(shape=(None, 10193), dtype=float32)

In [7]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(y_train[1:])
print(type(y_train))
print(y_train.dtype)

(683, 10193)
(683,)
(151, 10193)
(151,)
[1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 0 0 1 1 1 1 1 1 1 1 0 0 1 1 0 0
 0 0 0 0 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 0 1 0 1 0 1 0 0 1 1 1 0 0 0 0 1 1 1
 1 1 1 1 1 0 0 1 1 1 0 1 0 1 0 1 0 1 0 1 1 1 0 0 1 0 0 1 1 1 0 0 0 1 1 1 1
 0 0 0 1 1 1 0 1 1 1 0 0 1 1 1 0 1 0 0 1 1 0 0 0 0 1 1 0 1 1 1 0 0 1 1 0 0
 0 1 1 1 1 1 1 0 1 0 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 0 0 1
 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 1 1 0 1 1 1 1 0 1 0 1 0 0 1 1 0 1 0 1 1
 1 1 0 0 1 1 0 1 1 0 1 0 0 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 0 0 1 1 1 0 1 0 0
 0 1 1 0 1 1 1 1 0 0 1 1 1 1 1 0 1 0 0 0 1 1 0 0 1 0 0 0 0 1 1 0 0 1 1 0 0
 1 1 0 1 0 1 1 0 1 1 1 1 1 1 0 0 1 1 1 0 1 1 0 1 1 0 0 1 1 0 1 0 1 1 0 0 0
 0 1 0 0 0 0 1 0 0 1 0 1 0 0 1 1 1 1 0 0 1 1 1 1 0 1 1 1 1 0 0 0 1 0 0 0 0
 0 0 1 0 0 1 1 1 0 0 1 1 0 0 1 0 1 1 0 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0
 1 1 1 1 1 1 1 0 1 1 1 0 1 1 0 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 0 1 0 0 0 1 1
 0 1 0 0 0 0 1 1 1 1 1 0 1 0 1 0 1 1 0 1 0 0 0 1 0 1 1 0 1 0

In [9]:
# Ensure the device is set to CPU and import necessary modules
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

device = torch.device('cpu')

# Define the PyTorch Autoencoder model class
class Auto_encoder_model(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.input_fc = nn.Linear(input_dim, 5000)
        self.hidden_fc = nn.Linear(5000, 4000)
        self.hidden_fc2 = nn.Linear(4000, 3000)
        self.hidden_fc9 = nn.Linear(3000, 2000)
        self.hidden_fc10 = nn.Linear(2000, 1000)
        self.hidden_fc11 = nn.Linear(1000, 500)
        self.hidden_fc12 = nn.Linear(500, 200)
        self.hidden_fc13 = nn.Linear(200, 50)
        self.output_fc = nn.Linear(50, output_dim)
        self.dropout = nn.Dropout(0.2)
        self.dropout_mid = nn.Dropout(0.05)
        self.dropout_small = nn.Dropout(0.05)

    def forward(self, x):
        batch_size = x.shape[0]
        x = x.view(batch_size, -1)
        x1 = nn.functional.relu(self.input_fc(x))
        x1 = self.dropout_mid(x1)
        x1 = nn.functional.relu(self.hidden_fc(x1))
        x1 = self.dropout_mid(x1)
        x1 = nn.functional.relu(self.hidden_fc2(x1))
        x1 = self.dropout_mid(x1)
        x1 = nn.functional.relu(self.hidden_fc9(x1))
        x1 = self.dropout_mid(x1)
        x1 = nn.functional.relu(self.hidden_fc10(x1))
        x1 = self.dropout_mid(x1)
        x1 = nn.functional.relu(self.hidden_fc11(x1))
        x1 = self.dropout_small(x1)
        x1 = nn.functional.relu(self.hidden_fc12(x1))
        x1 = self.dropout_small(x1)
        x1 = nn.functional.relu(self.hidden_fc13(x1))
        x1 = self.dropout_small(x1)
        x1 = self.output_fc(x1)
        return x1

# Load dataset
with open("dataset_amyloid.pickle", 'rb') as f:
    X_train, y_train, X_test, y_test = pickle.load(f)

# Convert data to torch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).to(device)

# Define model parameters
input_dim = X_train.shape[1]
output_dim = y_train.shape

# Create the model instance
autoencoder_model = Auto_encoder_model(input_dim, output_dim).to(device)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder_model.parameters(), lr=0.001)

# Training the model
num_epochs = 10
batch_size = 64

# Create DataLoader for batching
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

for epoch in range(num_epochs):
    autoencoder_model.train()
    for batch_X, batch_y in train_loader:
        # Forward pass
        outputs = autoencoder_model(batch_X)
        loss = criterion(outputs, batch_y)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluating the model
autoencoder_model.eval()
with torch.no_grad():
    test_outputs = autoencoder_model(X_test_tensor)
    test_loss = criterion(test_outputs, y_test_tensor)
    print(f'Test Loss: {test_loss.item():.4f}')


TypeError: empty() received an invalid combination of arguments - got (tuple, dtype=NoneType, device=NoneType), but expected one of:
 * (tuple of ints size, *, tuple of names names, torch.memory_format memory_format, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, torch.memory_format memory_format, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
